# ⚡ Batch Processing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artiso-ai/dppvalidator/blob/main/examples/notebooks/03_batch_processing.ipynb)

Learn how to validate multiple DPPs efficiently using async batch processing.

This notebook covers:
- Async validation for high throughput
- Batch validation with concurrency control
- Processing directories of DPP files

In [ ]:
!pip install -q dppvalidator

## 📊 Sample Data

Let's create multiple DPPs to validate:

In [ ]:
# Generate sample DPPs
def create_sample_dpp(product_id: int, recycled_content: float) -> dict:
    return {
        "type": ["DigitalProductPassport", "VerifiableCredential"],
        "@context": [
            "https://www.w3.org/ns/credentials/v2",
            "https://test.uncefact.org/vocabulary/untp/dpp/0.6.1/",
        ],
        "id": f"https://example.com/credentials/dpp-{product_id:04d}",
        "issuer": {"id": "https://example.com/issuers/001", "name": "Batch Co."},
        "credentialSubject": {
            "type": ["Product"],
            "id": f"https://example.com/products/{product_id:04d}",
            "name": f"Product {product_id}",
            "circularityScorecard": {
                "type": ["CircularityScorecard"],
                "recycledContent": recycled_content,
                "recyclableContent": 0.90,
            },
        },
    }


# Create 100 sample DPPs with varying recycled content
import random

random.seed(42)

sample_dpps = [create_sample_dpp(i, random.uniform(0.1, 0.8)) for i in range(100)]

print(f"Created {len(sample_dpps)} sample DPPs")

## 🔄 Synchronous Batch Validation

Simple approach - validate one at a time:

In [ ]:
import time

from dppvalidator.validators import ValidationEngine

engine = ValidationEngine()

# Synchronous validation
start = time.perf_counter()
sync_results = [engine.validate(dpp) for dpp in sample_dpps]
sync_time = time.perf_counter() - start

valid_count = sum(1 for r in sync_results if r.valid)
print(f"Sync validation: {len(sample_dpps)} DPPs in {sync_time:.3f}s")
print(f"Valid: {valid_count}/{len(sample_dpps)}")
print(f"Throughput: {len(sample_dpps) / sync_time:.0f} DPPs/second")

## ⚡ Async Batch Validation

Use `validate_batch` for concurrent validation:

In [ ]:
import asyncio


async def run_batch_validation():
    start = time.perf_counter()
    results = await engine.validate_batch(sample_dpps, concurrency=10)
    elapsed = time.perf_counter() - start
    return results, elapsed


# Run async batch validation
async_results, async_time = await run_batch_validation()

valid_count = sum(1 for r in async_results if r.valid)
print(f"Async validation: {len(sample_dpps)} DPPs in {async_time:.3f}s")
print(f"Valid: {valid_count}/{len(sample_dpps)}")
print(f"Throughput: {len(sample_dpps) / async_time:.0f} DPPs/second")

## 🎚️ Concurrency Tuning

Test different concurrency levels to find optimal throughput:

In [ ]:
async def benchmark_concurrency(concurrency: int):
    start = time.perf_counter()
    results = await engine.validate_batch(sample_dpps, concurrency=concurrency)
    elapsed = time.perf_counter() - start
    throughput = len(sample_dpps) / elapsed
    return concurrency, elapsed, throughput


# Test different concurrency levels
print("Concurrency | Time (s) | Throughput (DPPs/s)")
print("-" * 45)

for concurrency in [1, 5, 10, 20, 50]:
    c, t, tp = await benchmark_concurrency(concurrency)
    print(f"{c:^11} | {t:^8.3f} | {tp:^18.0f}")

## 📁 Processing Files from Directory

In [ ]:
import json
from pathlib import Path

# Create temp directory with DPP files
temp_dir = Path("temp_dpps")
temp_dir.mkdir(exist_ok=True)

# Save first 10 DPPs as files
for i, dpp in enumerate(sample_dpps[:10]):
    (temp_dir / f"dpp_{i:04d}.json").write_text(json.dumps(dpp, indent=2))


# Validate all files in directory
def validate_directory(directory: Path) -> list:
    results = []
    for json_file in directory.glob("*.json"):
        result = engine.validate_file(json_file)
        results.append((json_file.name, result))
    return results


file_results = validate_directory(temp_dir)
print(f"Validated {len(file_results)} files:")
for filename, result in file_results:
    status = "✓" if result.valid else "✗"
    print(f"  {status} {filename}")

## 📊 Aggregating Results

In [ ]:
from collections import Counter

# Aggregate error codes across all results
all_errors = []
all_warnings = []

for result in sync_results:
    all_errors.extend([e.code for e in result.errors])
    all_warnings.extend([w.code for w in result.warnings])

error_counts = Counter(all_errors)
warning_counts = Counter(all_warnings)

print("Error Code Distribution:")
for code, count in error_counts.most_common(5):
    print(f"  {code}: {count}")

print("\nWarning Code Distribution:")
for code, count in warning_counts.most_common(5):
    print(f"  {code}: {count}")

## 🧹 Cleanup

In [ ]:
import shutil

shutil.rmtree(temp_dir, ignore_errors=True)
print("✓ Cleaned up temporary files")

## 🎉 Key Takeaways

- Use `validate_batch()` for high-throughput scenarios
- Tune `concurrency` based on your workload
- Aggregate results for bulk analysis
- dppvalidator can handle 80k+ validations/second